In [1]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.utils import np_utils
import numpy as np
import pandas as pd
from keras import initializers
from sklearn.model_selection import train_test_split
import requests
import datetime

def init_weights(shape, name=None):
    return initializers.normal(shape, scale=0.01, name=name)

Using TensorFlow backend.


In [2]:

API_ENDPOINT_URL =  'http://lauzhack.sqpub.ch/histo_prices'
r = requests.get(API_ENDPOINT_URL)
raw_data = r.text.split('\n')
data_series = pd.Series(raw_data)
data_series = data_series.str.split(' ')
data = pd.DataFrame(data_series.values.tolist(), index=data_series.index)
data.columns = ['timestamp', 'price']
data['timestamp'] = pd.to_datetime(data.timestamp)
time_stamps = data['timestamp']
data.set_index('timestamp', inplace=True)
data['price'] = data['price'].astype(float)
mean_minute = data.groupby(pd.Grouper(freq='60s')).mean()[2881:].dropna()
mean_minute


input_step_size = 50
output_size = 30
sliding_window = False

prices= data.loc[:,'price'].values
times = data.loc[:,'price'].values
prices.shape


# In[31]:


outputs = []
inputs = []
output_times = []
input_times = []
if sliding_window:
    for i in range(len(prices)-input_step_size-output_size):
        inputs.append(prices[i:i + input_step_size])
        input_times.append(times[i:i + input_step_size])
        outputs.append(prices[i + input_step_size: i + input_step_size+ output_size])
        output_times.append(times[i + input_step_size: i + input_step_size+ output_size])
else:
    for i in range(0,len(prices)-input_step_size-output_size, input_step_size):
        inputs.append(prices[i:i + input_step_size])
        input_times.append(times[i:i + input_step_size])
        outputs.append(prices[i + input_step_size: i + input_step_size+ output_size])
        output_times.append(times[i + input_step_size: i + input_step_size+ output_size])
inputs= np.array(inputs)
outputs= np.array(outputs)
output_times = np.array(output_times)
input_times = np.array(input_times)


In [3]:
import numpy as np
import pandas as pd
 
class PastSampler:
    '''
    Forms training samples for predicting future values from past value
    '''
     
    def __init__(self, N, K, sliding_window = True):
        '''
        Predict K future sample using N previous samples
        '''
        self.K = K
        self.N = N
        self.sliding_window = sliding_window
 
    def transform(self, A):
        M = self.N + self.K     #Number of samples per row (sample + target)
        #indexes
        if self.sliding_window:
            I = np.arange(M) + np.arange(A.shape[0] - M + 1).reshape(-1, 1)
        else:
            if A.shape[0]%M == 0:
                I = np.arange(M)+np.arange(0,A.shape[0],M).reshape(-1,1)
                
            else:
                I = np.arange(M)+np.arange(0,A.shape[0] -M,M).reshape(-1,1)
            
        B = A[I].reshape(-1, M * A.shape[1], A.shape[2])
        ci = self.N * A.shape[1]    #Number of features per sample
        return B[:, :ci], B[:, ci:] #Sample matrix, Target matrix



#Columns of price data to use
columns = ['price']
data = data.loc[:,columns]
original_data = data.loc[:,columns]

In [4]:

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# normalization
for c in columns:
    data[c] = scaler.fit_transform(data[c].values.reshape(-1,1))
    
    
    
#Features are input sample dimensions(channels)
A = np.array(data)[:,None,:]
original_A = np.array(original_data)[:,None,:]
time_stamps = np.array(time_stamps)[:,None,None]

#Make samples of temporal sequences of pricing data (channel)
NPS, NFS = 256, 16         #Number of past and future samples
ps = PastSampler(NPS, NFS, sliding_window=False)
B, Y = ps.transform(A)
input_times, output_times = ps.transform(time_stamps)
original_B, original_Y = ps.transform(original_A)

file_name='bitcoin2015to2017_close.h5'

import h5py
with h5py.File(file_name, 'w') as f:
    f.create_dataset("inputs", data = B)
    f.create_dataset('outputs', data = Y)
    f.create_dataset("input_times", data = np.string_(input_times))
    f.create_dataset('output_times', data = np.string_(output_times))
    f.create_dataset("original_datas", data=np.array(original_data))
    f.create_dataset('original_inputs',data=original_B)
    f.create_dataset('original_outputs',data=original_Y)



In [5]:
#CNN

import pandas as pd
import numpy as numpy
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv1D, MaxPooling1D, LeakyReLU, PReLU
from keras.utils import np_utils
from keras.callbacks import CSVLogger, ModelCheckpoint
import h5py
import os
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session




with h5py.File(''.join(['bitcoin2015to2017_close.h5']), 'r') as hf:
    datas = hf['inputs'].value
    labels = hf['outputs'].value


output_file_name='bitcoin2015to2017_close_CNN_2_relu'

step_size = datas.shape[1]
batch_size= 8
nb_features = datas.shape[2]

epochs = 100

#split training validation
training_size = int(0.8* datas.shape[0])
training_datas = datas[:training_size,:]
training_labels = labels[:training_size,:]
validation_datas = datas[training_size:,:]
validation_labels = labels[training_size:,:]
#build model

# 2 layers
model = Sequential()


model.add(Conv1D(activation='relu', input_shape=(step_size, nb_features), strides=3, filters=8, kernel_size=20))
model.add(Dropout(0.5))
model.add(Conv1D( strides=4, filters=nb_features, kernel_size=16))

'''
# 3 Layers
model.add(Conv1D(activation='relu', input_shape=(step_size, nb_features), strides=3, filters=8, kernel_size=8))
#model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Conv1D(activation='relu', strides=2, filters=8, kernel_size=8))
#model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Conv1D( strides=2, filters=nb_features, kernel_size=8))
# 4 layers
model.add(Conv1D(activation='relu', input_shape=(step_size, nb_features), strides=2, filters=8, kernel_size=2))
#model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Conv1D(activation='relu', strides=2, filters=8, kernel_size=2))
#model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Conv1D(activation='relu', strides=2, filters=8, kernel_size=2))
#model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Conv1D( strides=2, filters=nb_features, kernel_size=2))
'''

model.compile(loss='mse', optimizer='adam')
model.fit(training_datas, training_labels,verbose=1, batch_size=batch_size,validation_data=(validation_datas,validation_labels), epochs = epochs, callbacks=[CSVLogger(output_file_name+'.csv', append=True),ModelCheckpoint('weights/'+output_file_name+'-{epoch:02d}-{val_loss:.5f}.hdf5', monitor='val_loss', verbose=1,mode='min')])

Train on 22 samples, validate on 6 samples
Epoch 1/100
22/22 [==============================] - 0s 9ms/step - loss: 0.3511 - val_loss: 0.0162

Epoch 00001: saving model to weights/bitcoin2015to2017_close_CNN_2_relu-01-0.01620.hdf5
Epoch 2/100
22/22 [==============================] - 0s 231us/step - loss: 0.2019 - val_loss: 0.0082

Epoch 00002: saving model to weights/bitcoin2015to2017_close_CNN_2_relu-02-0.00816.hdf5
Epoch 3/100
22/22 [==============================] - 0s 418us/step - loss: 0.1415 - val_loss: 0.0033

Epoch 00003: saving model to weights/bitcoin2015to2017_close_CNN_2_relu-03-0.00327.hdf5
Epoch 4/100
22/22 [==============================] - 0s 301us/step - loss: 0.0980 - val_loss: 0.0012

Epoch 00004: saving model to weights/bitcoin2015to2017_close_CNN_2_relu-04-0.00123.hdf5
Epoch 5/100
22/22 [==============================] - 0s 230us/step - loss: 0.0667 - val_loss: 0.0014

Epoch 00005: saving model to weights/bitcoin2015to2017_close_CNN_2_relu-05-0.00138.hdf5
Epoch 6/1

Epoch 44/100
22/22 [==============================] - 0s 422us/step - loss: 0.0145 - val_loss: 0.0017

Epoch 00044: saving model to weights/bitcoin2015to2017_close_CNN_2_relu-44-0.00168.hdf5
Epoch 45/100
22/22 [==============================] - 0s 304us/step - loss: 0.0115 - val_loss: 0.0015

Epoch 00045: saving model to weights/bitcoin2015to2017_close_CNN_2_relu-45-0.00145.hdf5
Epoch 46/100
22/22 [==============================] - 0s 292us/step - loss: 0.0103 - val_loss: 0.0015

Epoch 00046: saving model to weights/bitcoin2015to2017_close_CNN_2_relu-46-0.00148.hdf5
Epoch 47/100
22/22 [==============================] - 0s 288us/step - loss: 0.0116 - val_loss: 0.0016

Epoch 00047: saving model to weights/bitcoin2015to2017_close_CNN_2_relu-47-0.00164.hdf5
Epoch 48/100
22/22 [==============================] - 0s 354us/step - loss: 0.0157 - val_loss: 0.0019

Epoch 00048: saving model to weights/bitcoin2015to2017_close_CNN_2_relu-48-0.00192.hdf5
Epoch 49/100
22/22 [=========================

22/22 [==============================] - 0s 371us/step - loss: 0.0085 - val_loss: 0.0017

Epoch 00087: saving model to weights/bitcoin2015to2017_close_CNN_2_relu-87-0.00169.hdf5
Epoch 88/100
22/22 [==============================] - 0s 255us/step - loss: 0.0118 - val_loss: 0.0016

Epoch 00088: saving model to weights/bitcoin2015to2017_close_CNN_2_relu-88-0.00165.hdf5
Epoch 89/100
22/22 [==============================] - 0s 298us/step - loss: 0.0103 - val_loss: 0.0017

Epoch 00089: saving model to weights/bitcoin2015to2017_close_CNN_2_relu-89-0.00166.hdf5
Epoch 90/100
22/22 [==============================] - 0s 350us/step - loss: 0.0106 - val_loss: 0.0021

Epoch 00090: saving model to weights/bitcoin2015to2017_close_CNN_2_relu-90-0.00206.hdf5
Epoch 91/100
22/22 [==============================] - 0s 314us/step - loss: 0.0119 - val_loss: 0.0017

Epoch 00091: saving model to weights/bitcoin2015to2017_close_CNN_2_relu-91-0.00174.hdf5
Epoch 92/100
22/22 [==============================] - 0s 2

In [6]:
#LSTM

import pandas as pd
import numpy as numpy
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,Reshape
from keras.layers import Conv1D, MaxPooling1D
from keras.utils import np_utils
from keras.layers import LSTM, LeakyReLU
from keras.callbacks import CSVLogger, ModelCheckpoint
import h5py
import os
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session




with h5py.File(''.join(['bitcoin2015to2017_close.h5']), 'r') as hf:
    datas = hf['inputs'].value
    labels = hf['outputs'].value




step_size = datas.shape[1]
units= 50
second_units = 30
batch_size = 8
nb_features = datas.shape[2]
epochs = 100
output_size=16
output_file_name='bitcoin2015to2017_close_LSTM_1_tanh_leaky_'
#split training validation
training_size = int(0.8* datas.shape[0])
training_datas = datas[:training_size,:]
training_labels = labels[:training_size,:,0]
validation_datas = datas[training_size:,:]
validation_labels = labels[training_size:,:,0]


#build model
model = Sequential()
model.add(LSTM(units=units,activation='tanh', input_shape=(step_size,nb_features),return_sequences=False))
model.add(Dropout(0.8))
model.add(Dense(output_size))
model.add(LeakyReLU())
model.compile(loss='mse', optimizer='adam')
model.fit(training_datas, training_labels, batch_size=batch_size,validation_data=(validation_datas,validation_labels), epochs = epochs, callbacks=[CSVLogger(output_file_name+'.csv', append=True),ModelCheckpoint('weights/'+output_file_name+'-{epoch:02d}-{val_loss:.5f}.hdf5', monitor='val_loss', verbose=1,mode='min')])

Train on 22 samples, validate on 6 samples
Epoch 1/100
22/22 [==============================] - 1s 43ms/step - loss: 0.1773 - val_loss: 0.0157

Epoch 00001: saving model to weights/bitcoin2015to2017_close_LSTM_1_tanh_leaky_-01-0.01570.hdf5
Epoch 2/100
22/22 [==============================] - 0s 13ms/step - loss: 0.1807 - val_loss: 0.0136

Epoch 00002: saving model to weights/bitcoin2015to2017_close_LSTM_1_tanh_leaky_-02-0.01360.hdf5
Epoch 3/100
22/22 [==============================] - 0s 13ms/step - loss: 0.1683 - val_loss: 0.0112

Epoch 00003: saving model to weights/bitcoin2015to2017_close_LSTM_1_tanh_leaky_-03-0.01123.hdf5
Epoch 4/100
22/22 [==============================] - 0s 13ms/step - loss: 0.1590 - val_loss: 0.0088

Epoch 00004: saving model to weights/bitcoin2015to2017_close_LSTM_1_tanh_leaky_-04-0.00883.hdf5
Epoch 5/100
22/22 [==============================] - 0s 12ms/step - loss: 0.1393 - val_loss: 0.0066

Epoch 00005: saving model to weights/bitcoin2015to2017_close_LSTM_1_

22/22 [==============================] - 0s 12ms/step - loss: 0.0389 - val_loss: 7.1501e-04

Epoch 00042: saving model to weights/bitcoin2015to2017_close_LSTM_1_tanh_leaky_-42-0.00072.hdf5
Epoch 43/100
22/22 [==============================] - 0s 12ms/step - loss: 0.0449 - val_loss: 7.0982e-04

Epoch 00043: saving model to weights/bitcoin2015to2017_close_LSTM_1_tanh_leaky_-43-0.00071.hdf5
Epoch 44/100
22/22 [==============================] - 0s 15ms/step - loss: 0.0638 - val_loss: 8.1450e-04

Epoch 00044: saving model to weights/bitcoin2015to2017_close_LSTM_1_tanh_leaky_-44-0.00081.hdf5
Epoch 45/100
22/22 [==============================] - 0s 15ms/step - loss: 0.0543 - val_loss: 9.2658e-04

Epoch 00045: saving model to weights/bitcoin2015to2017_close_LSTM_1_tanh_leaky_-45-0.00093.hdf5
Epoch 46/100
22/22 [==============================] - 0s 13ms/step - loss: 0.0458 - val_loss: 9.7944e-04

Epoch 00046: saving model to weights/bitcoin2015to2017_close_LSTM_1_tanh_leaky_-46-0.00098.hdf5
Epo

Epoch 83/100
22/22 [==============================] - 1s 23ms/step - loss: 0.0375 - val_loss: 0.0012

Epoch 00083: saving model to weights/bitcoin2015to2017_close_LSTM_1_tanh_leaky_-83-0.00116.hdf5
Epoch 84/100
22/22 [==============================] - 1s 24ms/step - loss: 0.0244 - val_loss: 0.0017

Epoch 00084: saving model to weights/bitcoin2015to2017_close_LSTM_1_tanh_leaky_-84-0.00173.hdf5
Epoch 85/100
22/22 [==============================] - 1s 25ms/step - loss: 0.0343 - val_loss: 0.0025

Epoch 00085: saving model to weights/bitcoin2015to2017_close_LSTM_1_tanh_leaky_-85-0.00249.hdf5
Epoch 86/100
22/22 [==============================] - 1s 25ms/step - loss: 0.0338 - val_loss: 0.0028

Epoch 00086: saving model to weights/bitcoin2015to2017_close_LSTM_1_tanh_leaky_-86-0.00276.hdf5
Epoch 87/100
22/22 [==============================] - 1s 26ms/step - loss: 0.0271 - val_loss: 0.0027

Epoch 00087: saving model to weights/bitcoin2015to2017_close_LSTM_1_tanh_leaky_-87-0.00272.hdf5
Epoch 88/1

In [7]:
#Plot

from keras import applications
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.callbacks import CSVLogger
import tensorflow as tf
from scipy.ndimage import imread
import numpy as np
import random
from keras.layers import LSTM
from keras.layers import Conv1D, MaxPooling1D, LeakyReLU
from keras import backend as K
import keras
from keras.callbacks import CSVLogger, ModelCheckpoint
from keras.backend.tensorflow_backend import set_session
from keras import optimizers
import h5py
from sklearn.preprocessing import MinMaxScaler
import os
import pandas as pd
# import matplotlib

import matplotlib.pyplot as plt

os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

with h5py.File(''.join(['bitcoin2015to2017_close.h5']), 'r') as hf:
    datas = hf['inputs'].value
    labels = hf['outputs'].value
    input_times = hf['input_times'].value
    output_times = hf['output_times'].value
    original_inputs = hf['original_inputs'].value
    original_outputs = hf['original_outputs'].value
    original_datas = hf['original_datas'].value
    

scaler=MinMaxScaler()
#split training validation
training_size = int(0.8* datas.shape[0])
training_datas = datas[:training_size,:,:]
training_labels = labels[:training_size,:,:]
validation_datas = datas[training_size:,:,:]
validation_labels = labels[training_size:,:,:]
validation_original_outputs = original_outputs[training_size:,:,:]
validation_original_inputs = original_inputs[training_size:,:,:]
validation_input_times = input_times[training_size:,:,:]
validation_output_times = output_times[training_size:,:,:]

ground_true = np.append(validation_original_inputs,validation_original_outputs, axis=1)
ground_true_times = np.append(validation_input_times,validation_output_times, axis=1)
step_size = datas.shape[1]
batch_size= 8
nb_features = datas.shape[2]

model = Sequential()

# 2 layers
model.add(Conv1D(activation='relu', input_shape=(step_size, nb_features), strides=3, filters=8, kernel_size=20))
# model.add(LeakyReLU())
model.add(Dropout(0.25))
model.add(Conv1D( strides=4, filters=nb_features, kernel_size=16))
model.load_weights('weights/bitcoin2015to2017_close_CNN_2_relu-44-0.00166.hdf5')
model.compile(loss='mse', optimizer='adam')

In [19]:
predicted = model.predict(validation_datas)
predicted_inverted = []

for i in range(original_datas.shape[1]):
    scaler.fit(original_datas[:,i].reshape(-1,1))
    predicted_inverted.append(scaler.inverse_transform(predicted[:,:,i]))
print(np.array(predicted_inverted).shape)
#get only the close data
ground_true = ground_true[:,:,0].reshape(-1)
ground_true_times = ground_true_times.reshape(-1)

ground_true_times = pd.to_datetime(ground_true_times, unit='s')
# since we are appending in the first dimension
predicted_inverted = np.array(predicted_inverted)[0,:,:].reshape(-1)
print(np.array(predicted_inverted).shape)
validation_output_times = pd.to_datetime(validation_output_times.reshape(-1), unit='s')

(1, 6, 16)


IndexError: too many indices for array